In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from tensorflow.python.client import device_lib
#print device_lib.list_local_devices()

In [10]:
%matplotlib inline
import tensorflow as tf
import numpy as np
from pdrnn import general_logpmf, pmf_entropy
from StochasticDilateNet import StochasticDilateNet
from scipy.stats import bernoulli
import hdf5storage as hdst
from time import clock
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
from subset_data import subset_data

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
def make_sequence(T, batch_size = 64):
    head = np.random.randint(1, 8, size=(10,batch_size,1))
    body = np.full([T-1,batch_size,1], 8)
    tail = np.full([11,batch_size,1], 9)
    return np.concatenate((head, body, tail), axis=0)

seq =  make_sequence(T=2000)


In [15]:
inputs_train, target_train = mnist.train.next_batch(500)
tiny_mnist = subset_data(inputs_train, target_train)

#X, Y = mnist.train.next_batch(batch_size)
def make_noisy_mnist(X, Y, T):
    X = np.expand_dims(np.transpose(X), axis=2)
    noisy = np.random.uniform(size=(T-784,X.shape[1],1))
    return (np.concatenate((X, noisy), axis=0), Y[np.newaxis,:]) 

In [ ]:

inputs_train, target_train = make_noisy_mnist(1000, batch_size=64)
inputs_train.shape

In [ ]:
#hdst.savemat('/mnt/hdd1/kqian3/rl_struct/seq',{'seq':seq})
data = hdst.loadmat('/mnt/hdd1/kqian3/rl_struct/seq')
seq = data['seq']

In [11]:
tf.reset_default_graph()

support = [[1,197],[1,296],[1,444],[1,666],[1,1000]]
n_layers = len(support)
hidden_structs = [20]*n_layers
lambda_b = 0.9
model = StochasticDilateNet(hidden_structs,
                            support,
                            n_layers=n_layers,
                            n_classes=10,
                            n_evaluate=1,
                            input_dims=1,
                            cell_type="RNN")
# define session
sess_config = tf.ConfigProto(allow_soft_placement=True,
                             log_device_placement=False,
                             intra_op_parallelism_threads=20,
                             inter_op_parallelism_threads=4)
sess_config.gpu_options.allow_growth = True

In [34]:
model.struct_train_ops[3] = tf.no_op('stop')

In [35]:
model.struct_train_ops

[<tf.Operation 'NoOp_1' type=NoOp>,
 <tf.Operation 'Adam_2' type=NoOp>,
 <tf.Operation 'Adam_3' type=NoOp>,
 <tf.Operation 'stop' type=NoOp>,
 <tf.Operation 'Adam_5' type=NoOp>]

In [30]:
X, Y = mnist.train.next_batch(64)
inputs_test, target_test = make_noisy_mnist(X, Y, 1000)
feed_dict = {d: r for d, r in zip(model.selections, [5, 238, 43, 72, 72])}  
feed_dict[model.inputs] = inputs_test
feed_dict[model.labels] = target_test
masks = [np.full((1,support[i][1]),-np.inf) for i in xrange(n_layers)]
masks[4][0,100] = 0.0
log_prob = general_logpmf(100,masks[4])
entropy = pmf_entropy(masks[4])
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    print sess.run(log_prob)
    print sess.run(tf.nn.softmax(masks[4]))
    
    print sess.run(model.struct_vars[4], 
                   feed_dict={model.struct_vars[4]:masks[4]})
    
    print sess.run(model.entropy,
                   feed_dict={model.struct_vars[4]:masks[4]})
  
    a = clock()
    for _ in range(100):
        sess.run(model.struct_train_ops, feed_dict=feed_dict)
    print clock()-a

0.0
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

7.941647


In [8]:
import tensorflow as tf
tf.reset_default_graph()
x = tf.get_variable('input', dtype=tf.float32, initializer=tf.constant(1.0))
y = x * x
y_ = tf.placeholder(shape=(), dtype=tf.float32)
fixed = tf.placeholder_with_default(False, shape=())
z = tf.where(tf.fill(y.get_shape(), fixed), y_, y)

optimizer = tf.train.GradientDescentOptimizer(0.01)
minimize = optimizer.minimize(z)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print sess.run(x)
    
    sess.run(minimize)
    
    print sess.run(x)
    
    sess.run(minimize, feed_dict={fixed:False, y_:1.0})

    
    print sess.run(x)

1.0
0.98
0.96040004


In [ ]:
batch_size = 4096
X, Y = mnist.validation.next_batch(batch_size)
inputs_valid, target_valid = make_noisy_mnist(X, Y, 2000, batch_size)
feed_dict = {d: r for d, r in zip(model.dilations, [231,130])}  
feed_dict[model.inputs] = inputs_valid
feed_dict[model.labels] = target_valid

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start = clock()
    print sess.run(model.logits, feed_dict=feed_dict).shape
    print sess.run(model.labels, feed_dict=feed_dict).shape
    print clock()-start 

In [38]:
def list_enqueue(x, lst, L=10):
    if len(lst) == L:
        lst.pop(0)
        lst.append(x)
    else:
        lst.append(x)
        
def find_closest_element(myNumber, lst):
    a = min(lst[::-1], key=lambda x:abs(x-myNumber))
    k = len(lst)-1-lst[::-1].index(a)
    return k

def list_enqueue_batch(xs, lsts):
    for i, x in enumerate(xs):
        list_enqueue(x, lsts[i])
        
def find_closest_element_batch(xs, lsts):
    ks = []
    for i, x in enumerate(xs):
        k = find_closest_element(x, lsts[i])
        ks.append(k)
    return ks   

def restore_layers(savers, ks, logdir, bottom=0):
    for i in xrange(bottom, n_layers):
        path = "{}model-{}".format(logdir, 
                                   history_steps[ks[i]])
        savers[i].restore(sess, path)  

In [ ]:
try:
    sess.close()
except:
    pass

# [5, 238, 43, 72]
# [5, 266, 45, 63]
# [8, 268, 125, 136]
# [8, 39, 51, 115]
sess = tf.Session(config=sess_config)
# initialize all the parameters
sess.run(tf.global_variables_initializer())

feed_dict = {d: r for d, r in zip(model.dilations, [5, 238, 43, 72])}  

batch_size = 64
losses = []

start = clock()
for step in range(3000):
    X, Y = mnist.train.next_batch(batch_size)
    inputs_test, target_test = make_noisy_mnist(X, Y, 1000, batch_size)
    feed_dict[model.inputs] = inputs_test
    feed_dict[model.labels] = target_test
        
    _, loss_value, accuracy = sess.run([model.weights_train_op, model.loss_for_w, model.accuracy], 
                                       feed_dict=feed_dict)
    loss_value = sess.run(model.loss_for_w, feed_dict=feed_dict)
    assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
    losses.append(loss_value)
    
    if step % 10 == 0:
        #model.saver_for_w.save(sess, "{}model".format('/mnt/hdd1/kqian3/rl_struct/'), global_step=step)
        print('Step {}, loss = {}, accuracy = {}'.format(step, loss_value, accuracy))
print clock()-start        

In [ ]:
l_1 = np.array(losses)

In [ ]:
plt.plot(l_1)
plt.plot(l_2)
plt.show()

In [ ]:
with tf.Session(config=sess_config) as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    model.saver_for_w.restore(sess, "{}model-100".format('/mnt/hdd1/kqian3/rl_struct/'))
    
    
    for step in range(1000):
        #seq = bernoulli.rvs(0.5, size=(100,32,1))
        feed_dict = {model.inputs: seq,
                     model.labels: seq[0,:,0]}     
        
        if np.mod(step, 10) < 5:
            _, loss_value = sess.run([model.weights_train_op, model.loss_for_w], 
                                     feed_dict=feed_dict)
        else:
            sess.run(model.struct_train_op, feed_dict=feed_dict)
        
        
        if step == 0:
            sess.run(tf.assign(model.b, loss_value))
        else:
            sess.run(tf.assign(model.b, (lambda_b*model.b+(1-lambda_b)*loss_value)))
        
        if step % 10 == 0:
            struct_params, rates, b, accuracy = \
            sess.run([model.struct_vars, model.rates, model.b, model.accuracy],
                    feed_dict=feed_dict)
            print('Step {}, struct params {},\n Rates {}, b = {}, accuracy = {}'.format(
                step, struct_params, rates, b, accuracy))
            
            #sess.run(model.struct_clip_op, feed_dict={model.inputs: seq})
            #struct_params = sess.run(model.struct_vars)
            #
            #print('After clip, struct params {}'.format(struct_params))

In [ ]:
try:
    sess.close()
except:
    pass
    
sess = tf.Session(config=sess_config)
# initialize all the parameters
sess.run(tf.global_variables_initializer())


num_w_op = 10
history_rates = [[] for _ in xrange(n_layers)]
history_steps = []

for step in xrange(100000):
    seq =  make_sequence(T=2000)      
    
    if step % num_w_op == 0:
        
        if step != 0:
            loss_ = sess.run(model.loss_for_w, feed_dict=feed_dict)
            if step == num_w_op:
                sess.run(tf.assign(model.b, loss_))
            else:
                sess.run(tf.assign(model.b, (lambda_b*model.b+(1-lambda_b)*loss_)))
                
            model.saver_for_w.save(sess, 
                                   "{}model".format('/mnt/hdd1/kqian3/rl_struct/'), 
                                   global_step=step)
            list_enqueue_batch(rates, history_rates)
            list_enqueue(step, history_steps)
                        
            
        if step >= 2*num_w_op:
            sess.run(model.struct_train_op, feed_dict=feed_dict)    
            
            
        picks = sess.run(model.picks)
        rates = sess.run(model.dilations, feed_dict={s: p for s, p in zip(model.selections, picks)})
        if len(history_steps) >= 1:
            ks = find_closest_element_batch(rates, history_rates)
            print 'Step {}, Draw = {},\n history_rates = {},\n Replace = {}'.format(step, rates, history_rates, ks)
            restore_layers(model.savers, ks)
        
        
    feed_dict = {d: r for d, r in zip(model.selections, picks)}    
    feed_dict[model.inputs] = seq
    feed_dict[model.labels] = seq[0:10,:,0]
        
    sess.run(model.weights_train_op, feed_dict=feed_dict)
    
    if step % num_w_op == num_w_op-1:
        struct_vars, b, loss_value, accuracy = \
        sess.run([model.struct_vars, 
                  model.b, model.loss_for_w, model.accuracy],
                 feed_dict=feed_dict)
        print('Step {}, Rates {}, b = {}, loss = {}, accuracy = {}\n'.format(
            step, rates, b, loss_value, accuracy))
        if step % 100 == 99:
            plt.plot(struct_vars[0][0])
            plt.show()
            plt.plot(struct_vars[1][0])
            plt.show()
        if loss_value < 1e-2:
            break

In [ ]:
pmf = sess.run(tf.nn.softmax(model.struct_vars[-1]), feed_dict=feed_dict_w)
hdst.savemat('/mnt/hdd1/kqian3/rl_struct/pmf1',{'pmf':pmf})

In [52]:
# noisy mnist

try:
    sess.close()
except:
    pass
    
sess = tf.Session(config=sess_config)
# initialize all the parameters
sess.run(tf.global_variables_initializer())

logdir = '/mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/'
bottom = 0
num_w_op = 10
history_rates = [[] for _ in xrange(n_layers)]
history_steps = []
feed_dict = {}

masks = [np.full((1,support[i][1]),-np.inf) for i in xrange(n_layers)]

for step in xrange(1000000):
    X, Y = mnist.train.next_batch(64)
    inputs_train, target_train = make_noisy_mnist(X, Y, 1000)    
    
    if step % num_w_op == 0:
        if step != 0:
            #X, Y = mnist.validation.next_batch(64)
            #inputs_valid, target_valid = make_noisy_mnist(X, Y, 2000, 64)
            #feed_dict[model.inputs] = inputs_valid
            #feed_dict[model.labels] = target_valid
            
            loss_ = sess.run(model.loss_for_w, feed_dict=feed_dict)
            if step == num_w_op:
                sess.run(tf.assign(model.b, loss_))
            else:
                sess.run(tf.assign(model.b, (lambda_b*model.b+(1-lambda_b)*loss_)))
                
            model.saver_for_w.save(sess, 
                                     "{}model".format(logdir), 
                                     global_step=step)
            list_enqueue_batch(rates, history_rates)
            list_enqueue(step, history_steps)
                        
            
        if step >= 2*num_w_op:
            sess.run(model.struct_train_ops, feed_dict=feed_dict)    
            
            
        picks = sess.run(model.picks, feed_dict=feed_dict)
        rates = sess.run(model.dilations, feed_dict={s: p for s, p in zip(model.selections, picks)})
        if len(history_steps) >= 1:
            ks = find_closest_element_batch(rates, history_rates)
            print 'Step {}, Draw = {},\n history_rates = {},\n Replace = {}'.format(step, rates, history_rates, ks)
            restore_layers(model.savers, ks, logdir, bottom)
            
        
    for d, r in zip(model.selections, picks):
        feed_dict[d] = r
    feed_dict[model.inputs] = inputs_train
    feed_dict[model.labels] = target_train
        
    sess.run(model.weights_train_op, feed_dict=feed_dict)
    
    if step % num_w_op == num_w_op-1:
        #feed_dict[model.inputs] = inputs_valid
        #feed_dict[model.labels] = target_valid
        struct_vars, entropy, b, loss_value, accuracy = \
        sess.run([model.struct_vars, model.entropy,
                  model.b, model.loss_for_w, model.accuracy],
                 feed_dict=feed_dict)
        print('\n Step {}, Rates {}, b = {}, loss = {}, accuracy = {}\n Entropy = {}\n'.format(
            step, rates, b, loss_value, accuracy, entropy))
        # check entropy
        for l in xrange(n_layers):
            if entropy[l] > 1.5 and not np.isnan(entropy[l]):
                model.struct_train_ops[l] = tf.no_op('Stop')
                masks[l][0,np.argmax(struct_vars[l])] = 0.0
                feed_dict[model.struct_vars[l]] = masks[l]
        # save logits        
        if step % 100 == 99:
            for k in xrange(bottom, n_layers):
                hdst.savemat('{}pmf_{}_{}'.format(logdir,step,k),
                             {'pmf':struct_vars[k][0]})
        if loss_value < 1e-6:
            break


 Step 9, Rates [111, 177, 13, 252, 570], b = 0.0, loss = 2.16296339035, accuracy = 0.1875
 Entropy = [5.283203, 5.6903586, 6.0958247, 6.50129, 6.907755]

Step 10, Draw = [1, 1, 1, 1, 1],
 history_rates = [[111], [177], [13], [252], [570]],
 Replace = [0, 0, 0, 0, 0]
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-10
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-10
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-10
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-10
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-10

 Step 19, Rates [1, 1, 1, 1, 1], b = 2.16296339035, loss = 2.31535315514, accuracy = 0.140625
 Entropy = [nan, nan, nan, nan, nan]

Step 20, Draw = [1, 1, 1, 1, 1],
 history_rates = [[111, 1], [177, 1], [13, 1], [252, 1], [570, 1]],
 Replace = [1

INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-110
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-110
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-110
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-110

 Step 119, Rates [1, 1, 1, 1, 1], b = 2.26350307465, loss = 2.31544446945, accuracy = 0.09375
 Entropy = [nan, nan, nan, nan, nan]

Step 120, Draw = [1, 1, 1, 1, 1],
 history_rates = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 Replace = [9, 9, 9, 9, 9]
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-120
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-120
INFO:tensorflow:Restoring parameters fr

Step 210, Draw = [1, 1, 1, 1, 1],
 history_rates = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 Replace = [9, 9, 9, 9, 9]
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-210
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-210
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-210
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-210
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-210

 Step 219, Rates [1, 1, 1, 1, 1], b = 2.28631687164, loss = 2.31641364098, accuracy = 0.046875
 Entropy = [nan, nan, nan, nan, nan]

Step 220, Draw = [1, 1, 1, 1, 1],
 history_rates = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1

INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-300
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-300
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-300

 Step 309, Rates [1, 1, 1, 1, 1], b = 2.29257798195, loss = 2.32418489456, accuracy = 0.171875
 Entropy = [nan, nan, nan, nan, nan]

Step 310, Draw = [1, 1, 1, 1, 1],
 history_rates = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 Replace = [9, 9, 9, 9, 9]
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-310
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-310
INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/noisy_mnist_5_1000/model-310
INFO:tensorflow:Restoring parameters f

KeyboardInterrupt: 

In [ ]:
#loss = []
for rate in range(0,525,5):
    if rate == 0: rate = 1
    with tf.Session(config=sess_config) as sess:
        # initialize all the parameters
        sess.run(tf.global_variables_initializer())
        
        for step in range(500):
            feed_dict = {model.inputs: seq,
                         model.labels: np.transpose(seq[0:10,:,0],[1,0]),
                         model.rates[-1]: rate}
            sess.run(model.weights_train_op, feed_dict=feed_dict)
                    
        loss_value = sess.run(model.loss_for_w, feed_dict=feed_dict)
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
        #loss.append(loss_value)
        print 'Rate = {}, loss = {}'.format(rate, loss_value)    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(loss))
plt.show()

In [ ]:
hdst.savemat('/mnt/hdd1/kqian3/rl_struct/loss1_1',{'loss':np.array(loss)})

In [ ]:
print feed_dict_w[model.rates[-1]]
feed_dict_w[model.b] = 1.3
with tf.Session() as sess:
    # initialize all the parameters
    sess.run(tf.global_variables_initializer())
    
    model.saver_for_w.restore(sess, "{}model-100".format('/mnt/hdd1/kqian3/rl_struct/'))
    
    print sess.run(model.loss_per_example, feed_dict = feed_dict_w)
    print sess.run(model.b, feed_dict = feed_dict_w)
    print sess.run(model.struct_param, feed_dict = feed_dict_w)
    
    print sess.run(tf.gradients(model.loss_for_pi, model.struct_param), feed_dict = feed_dict_w)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.gradients(model.loss_for_pi, model.struct_vars[1]), feed_dict = feed_dict))

In [ ]:
print model.sess.run(model.loss_per_example, feed_dict=feed_dict)

In [ ]:
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 3]])
with tf.Session():
    print tf.pad(t, paddings, constant_values=0.0).eval() 

In [ ]:
tf.reset_default_graph()
init_params=[(1.0,0.5),(2.0,2.0)]
dilations = []
for l in xrange(2):
    with tf.variable_scope('struct_layer_{}'.format(l)):
        mu = tf.get_variable('mu', initializer=tf.constant(init_params[l][0]))
        sigma = tf.get_variable('sigma', initializer=tf.constant(init_params[l][1]))
                
    rates = tf.range(tf.ceil(tf.maximum(mu-sigma,0.5)), 
                     tf.ceil(tf.minimum(mu+sigma,20)))
    probs = triangular_pmf(rates, mu, sigma, 20)
    probs = tf.expand_dims(probs, 0)
    idx = tf.multinomial(tf.log(probs), 1)
    rates = tf.cast(rates, tf.int32)
    dilations.append(rates[idx[0][0]])  

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(probs)

In [ ]:
a = tf.constant([1])
b = a;
c = b;

with tf.Session() as sess:
    print sess.run(c, feed_dict = {b: [2]})

In [ ]:
from pdrnn import drnn_classification, _contruct_cells
from pdrnn import dRNN, multi_dRNN_with_dilations
# build the network
tf.reset_default_graph()

dilations = []
for l in xrange(3):
    #d = tf.placeholder_with_default(tf.constant(l+1), [])
    d = tf.placeholder(tf.int32, [])
    dilations.append(d)

inputs = tf.placeholder(tf.float32, shape = [None, None, 1], name = 'inputs')
hidden_structs=[4]*3

# construct a list of cells
cells = _contruct_cells(hidden_structs=hidden_structs, cell_type='LSTM')

logits = multi_dRNN_with_dilations(cells, inputs, hidden_structs, dilations)
#logits = dRNN(cells[0], inputs, dilations[0], 1, scope='default')

#logits = drnn_classification(x=inputs,
#                               hidden_structs=hidden_structs,
#                               dilations=dilations,
#                               n_classes=2,
#                               input_dims=1,
#                               cell_type='RNN')


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    pred = sess.run(logits, feed_dict={inputs: np.random.uniform(size=[100,32,1])})
    print pred.shape

In [ ]:
from pdrnn import _construct_cells
tf.reset_default_graph()
empty_inputs = [tf.zeros([1,3]) for _ in xrange(3)]
    
cell = tf.nn.rnn_cell.BasicLSTMCell(10)
outputs, _ = tf.nn.static_rnn(cell, 
                              empty_inputs, 
                              dtype=tf.float32,
                              scope='policy_net')

In [ ]:
h=tf.stack(outputs, axis=1)
logpmfs = tf.layers.conv1d(inputs=h[:,1:,:],
                               filters=1000,
                               kernel_size=1,
                               kernel_initializer=tf.zeros_initializer(),
                               bias_initializer=tf.constant_initializer(0.001))

In [ ]:
logpmfs[:,1,:100]